# Tensor Decomposition
A "hands-on" tutorial with TensorLy

- [Tensorly](http://tensorly.org/stable/index.html) is a Python library that aims at making it _easy_ to work with tensors.
- It comprises a set of tools to perform **tensor decomposition**, **tensor learning** and **tensor algebra**

In [1]:
import tensorly as tl
from tensorly.decomposition import parafac
import numpy as np

# Creating a Tensor

Tensors can be represented in different ways. One possibility is to see them as juxtaposed matrices.

## Example
Let us consider a tensor $X$, defined by its frontal slices, $X_{1}$ and $X_{2}$.

We have

$$X_{1} = \begin{bmatrix}
0 & 2 & 4 & 6\\
8 & 10 & 12 & 14
\\ 16 & 18 & 20 & 22
\end{bmatrix}$$

and

$$X_{2} = \begin{bmatrix}
1 & 3 & 5 & 7\\
9 & 11 & 13 & 15
\\ 17 & 19 & 21 & 23
\end{bmatrix}.$$


We can easily obtain this as a [NumPy](https://numpy.org/) array:

In [2]:
X = tl.tensor(np.arange(24).reshape((3, 4, 2)), dtype = tl.float32)

In [3]:
X

array([[[ 0.,  1.],
        [ 2.,  3.],
        [ 4.,  5.],
        [ 6.,  7.]],

       [[ 8.,  9.],
        [10., 11.],
        [12., 13.],
        [14., 15.]],

       [[16., 17.],
        [18., 19.],
        [20., 21.],
        [22., 23.]]], dtype=float32)

It is possible to then view the frontal slices by fixing the last axis.

In [4]:
X[:, :, 0]

array([[ 0.,  2.,  4.,  6.],
       [ 8., 10., 12., 14.],
       [16., 18., 20., 22.]], dtype=float32)

and, of course, the back slice.

In [5]:
X[:, :, 1]

array([[ 1.,  3.,  5.,  7.],
       [ 9., 11., 13., 15.],
       [17., 19., 21., 23.]], dtype=float32)

# Different backends
In TensorLy, it is possible to set different backends.
In practice, this means that the set of tools exposed to the user is always the same, but the manner in which the various operations are computed changes.

For instance, we might be interested in using the [PyTorch](https://pytorch.org/) backend, to exploit the GPU by means of [CUDA](https://developer.nvidia.com/CUDA-zone)

In [6]:
type(X)

numpy.ndarray

In [7]:
tl.set_backend('pytorch')

In [8]:
X = tl.tensor(np.arange(24).reshape((3, 4, 2)), device='cuda:0')
type(X)

torch.Tensor

We will set it to NumPy for the rest of the tutorial though.

In [9]:
tl.set_backend("numpy")

In [10]:
X = tl.tensor(np.arange(24).reshape((3, 4, 2)))
type(X)

numpy.ndarray

# Basic Operations
## Unfolding

Unfolding consists in rewriting the tensor as a juxtaposition of matrices.

Notice that to maintain consistency with Python's zero-based index system, **we start from 0**.

We had $X$ defined by:

$$X_{1} = \begin{bmatrix}
0 & 2 & 4 & 6\\
8 & 10 & 12 & 14
\\ 16 & 18 & 20 & 22
\end{bmatrix}$$

and

$$X_{2} = \begin{bmatrix}
1 & 3 & 5 & 7\\
9 & 11 & 13 & 15
\\ 17 & 19 & 21 & 23
\end{bmatrix}.$$

The 0-mode unfolding of it is:

$$X_{\left[0\right]} = \begin{bmatrix}
0 & 1 & 2 & 3 & 4 & 5 & 6 & 7\\
8 & 9 & 10 & 11 & 12 & 13 & 14 & 15\\
16 & 17 & 18 & 19 & 20 & 21 & 22 & 23\\
\end{bmatrix}
$$

The 1-mode unfolding is:
$$X_{\left[1\right]} = \begin{bmatrix}
0 & 1 & 8 & 9 & 16 & 17\\
2 & 3 & 10 & 11 & 18 & 19\\
4 & 5 & 12 & 13 & 20 & 21\\
6 & 7 & 14 & 15 & 22 & 23\\
\end{bmatrix}
$$

And, of course, the 2-mode unfolding is along the last axis.
$$X_{\left[2\right]} = \begin{bmatrix}
0 & 2 & 4 & 6 & 8 & 10 & 12 & 4 & 16 & 18 & 20 & 22\\
1 & 3 & 5 & 7 & 9 & 11 & 13 & 15 & 17 & 19 & 21 & 23
\end{bmatrix}
$$

### TensorLy
We simply do:

In [11]:
tl.unfold(X, mode = 0)

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23]])

In [12]:
tl.unfold(X, mode = 1)

array([[ 0,  1,  8,  9, 16, 17],
       [ 2,  3, 10, 11, 18, 19],
       [ 4,  5, 12, 13, 20, 21],
       [ 6,  7, 14, 15, 22, 23]])

In [13]:
tl.unfold(X, mode = 2)

array([[ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22],
       [ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23]])

# Outer Product

We now compute the outer product between two arrays:

$$ x = \begin{bmatrix}
2 \\
5 \\
\end{bmatrix}
$$

and 

$$y = \begin{bmatrix}
9\\
10\\
\end{bmatrix}.
$$

In [14]:
x = tl.tensor(np.array([2, 5]))
x

array([2, 5])

In [15]:
y = tl.tensor(np.array([9, 10]))
y

array([ 9, 10])

We obtain

$$ X = x \otimes y = xy^{t} = \begin{bmatrix}18 & 20\\ 45 & 50 \end{bmatrix}.$$

In [16]:
X = tl.tenalg.tensor_dot(x, y)

In [17]:
X

array([[18, 20],
       [45, 50]])